In [71]:
import json
import os
import os.path as path
import six
from torch.utils.data import Dataset
import random

import numpy as np
from torchvision import transforms as torch_transform
import torch
from torch.autograd import Variable
import torch.nn.functional as F

from PIL import Image

DRESS_FOLDER = "meta/json/"


ID_NAME = "retrieval_dresses.json"
TEST_NAME = "test_pairs_dresses.json"
TRAIN_NAME = "train_pairs_dresses.json"
PHOTO_FILE = "photos/photos.txt"
PHOTO_DRESS_FILE = "photos/photos_dress.txt"
SEED = 17
EPOCH = 100
MARGIN = 0.3
CHECKPOINT_FILE = "checkpoint.pth.tar"

torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
FINAL_EMBEDDING_SIZE = 128

def read_json(file_name):
    with open(file_name) as f:
        obj = json.loads(f.readline())
    return obj

<h1>Step 1 upload dataset metadata and photos info</h1>

In [3]:
!git clone https://github.com/pumpikano/street2shop

Cloning into 'street2shop'...
remote: Counting objects: 9, done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 9 (delta 1), reused 9 (delta 1), pack-reused 0
Unpacking objects: 100% (9/9), done.
Checking connectivity... done.


In [4]:
!mkdir street2shop/images

In [10]:
!curl -O http://www.tamaraberg.com/street2shop/wheretobuyit/meta.zip
!curl http://www.tamaraberg.com/street2shop/wheretobuyit/photos.tar | tar x

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2777k  100 2777k    0     0   297k      0  0:00:09  0:00:09 --:--:--  346k


In [13]:
!unzip meta.zip
!rm meta.zip

Archive:  meta.zip
   creating: meta/
  inflating: meta/.DS_Store          
   creating: __MACOSX/
   creating: __MACOSX/meta/
  inflating: __MACOSX/meta/._.DS_Store  
   creating: meta/json/
  inflating: meta/json/.DS_Store     
   creating: __MACOSX/meta/json/
  inflating: __MACOSX/meta/json/._.DS_Store  
  inflating: meta/json/retrieval_bags.json  
  inflating: meta/json/retrieval_belts.json  
  inflating: meta/json/retrieval_dresses.json  
  inflating: meta/json/retrieval_eyewear.json  
  inflating: meta/json/retrieval_footwear.json  
  inflating: meta/json/retrieval_hats.json  
  inflating: meta/json/retrieval_leggings.json  
  inflating: meta/json/retrieval_outerwear.json  
  inflating: meta/json/retrieval_pants.json  
  inflating: meta/json/retrieval_skirts.json  
  inflating: meta/json/retrieval_tops.json  
  inflating: meta/json/test_pairs_bags.json  
  inflating: meta/json/test_pairs_belts.json  
  inflating: meta/json/test_pairs_dresses.json  
  inflating: meta/json/test_pai

 <h1>step 2 extract street photos only</h1>

In [18]:
!mkdir ../data
!mkdir ../data/images

mkdir: cannot create directory ‘../data/images’: File exists


In [1]:



def get_photo_ids(obj, retr_dict):
    photos = set()
    for item in obj:
        photos.add(item["photo"])
        product_photos = retr_dict[item["product"]]
        photos.update(product_photos)
    return photos



In [30]:
if __name__ == "__main__":
    id_to_photo_dirty = read_json(DRESS_FOLDER+ID_NAME)
    id_to_photo_clean = {}
    for item in id_to_photo_dirty:
        if item["product"] in id_to_photo_clean:
            id_to_photo_clean[item["product"]].append(item["photo"])
        else:
            id_to_photo_clean[item["product"]] = [item["photo"]]
    test_dress = read_json(DRESS_FOLDER + TEST_NAME)
    train_dress = read_json(DRESS_FOLDER + TRAIN_NAME)
    test_set = get_photo_ids(test_dress, id_to_photo_clean)
    train_set = get_photo_ids(train_dress, id_to_photo_clean)
    result_set = test_set|train_set

    with open(PHOTO_DRESS_FILE, "w") as new_ph_file:
        with open(PHOTO_FILE) as ph_file:
            for line in ph_file:
                id = int(line.split(",")[0])
                if id in result_set:
                    new_ph_file.write(line)

In [40]:
!python street2shop/download.py --urls photos/photos_dress.txt --image_dir ../../data/images

^C
Error in atexit._run_exitfuncs:
Traceback (most recent call last):
  File "/home/snake/anaconda3/envs/myenv/lib/python3.6/concurrent/futures/thread.py", line 40, in _python_exit
    t.join()
  File "/home/snake/anaconda3/envs/myenv/lib/python3.6/threading.py", line 1056, in join
    self._wait_for_tstate_lock()
  File "/home/snake/anaconda3/envs/myenv/lib/python3.6/threading.py", line 1072, in _wait_for_tstate_lock
    elif lock.acquire(block, timeout):
KeyboardInterrupt


In [7]:
import os
import os.path as path

PHOTO_FOLDER = "../data/images"

ALL_IMAGES  = [int(name.split(".")[0]) for name in os.listdir(PHOTO_FOLDER) if path.isfile(PHOTO_FOLDER+"/"+name)]



form target dataset

In [60]:
PHOTO_FOLDER = "../data/images"
META_FOLDER = "../data/meta"
from torch.utils.data import Dataset
import random

IMG_SIZE =299

def default_image_loader(path, crop=None,size=None):
    if size is None:
        size = (IMG_SIZE, IMG_SIZE)
    image=Image.open(path).convert('RGB')
    if crop is not None:
        image = image.crop(crop)
    image=image.resize(size)
    return image

def get_negative(positive, items):
    while True:
        rand_item = random.choice(items)
        if positive != rand_item:
            return rand_item
    

def get_image(photo_id, folder=PHOTO_FOLDER, crop=None,transform=None):
    photo_name = str(photo_id)
    zeros = "0"*(9- len(photo_name))
    extension = ".jpeg", ".png", ".gif"
    full_photo_name = None
    for ext in extension:
        full_photo_name = folder+"/"+zeros+photo_name+ext
        if path.isfile(full_photo_name):
            break
    else:
        raise Exception("No Image found")
    image = default_image_loader(full_photo_name, crop=crop)
    if transform is None:
        return image
    else:
        return transform(image)

class DressDataset(Dataset):
    """street2shop dress dataset."""

    def __init__(self, photos_folder_name=PHOTO_FOLDER, transform = None):
        self.triplets = None
        self.retrieval = None
        self.photos_folder_name = photos_folder_name
        self.transform = transform
    
    def parse_triplets(self, triplets_file_name, retrieval_file_name=DRESS_FOLDER+ID_NAME):
        self.triplets = read_json(triplets_file_name)
        self.retrieval = read_json(retrieval_file_name)
        self.retrieval ={it["product"]:it["photo"] for it in self.retrieval}
        all_photos = [item["photo"] for item in self.triplets]
        for item in self.triplets:
            positive_photo = self.retrieval[item["product"]]
            anchor_photo = item["photo"]
            if positive_photo not in ALL_IMAGES or anchor_photo not in ALL_IMAGES:
                continue
            item["positive"] = positive_photo
            item["anchor"] = anchor_photo
            del item["product"]
            del item["photo"]
            negative = get_negative(item["positive"], ALL_IMAGES)
            item["negative"] = negative
        
        for item in self.triplets[:]:
            if "positive" not in item:
                self.triplets.remove(item)
    
    def __len__(self):
        return len(self.triplets)

    def __getitem__(self, index):
        sample = {}
        triplet = self.triplets[index]
        sample["positive"] = get_image(triplet["positive"], transform = self.transform)
        sample["negative"] = get_image(triplet["negative"], transform = self.transform)
        bbox = triplet["bbox"]
        anchor = get_image(triplet["anchor"], crop=(bbox["left"],bbox["top"],bbox["left"]+bbox["width"],bbox["top"]+bbox["height"]),
                          transform=self.transform)
        sample["anchor"] = anchor
        return ((sample["positive"],triplet["positive"]), (sample["anchor"],triplet["anchor"]),
                (sample["negative"], triplet["negative"]))
    
    def get_sliced_copy(self, start, end):
        copied = DressDataset(photos_folder_name=self.photos_folder_name, transform=self.transform)
        copied.triplets = self.triplets[start:end]
        return copied
    
    def resample(self, embeddings, mapping, margin):
        new_triplets = []
        reverse_mapping = {v:k for k,v in six.iteritems(mapping)}
        for triplet in self.triplets:
            pos, anc, neg = mapping[triplet["positive"]], mapping[triplet["anchor"]], mapping[triplet["negative"]]
            dists = np.linalg.norm(embeddings - embeddings[anc], axis = 1)
            pos_dist = dists[pos]
            semihard_dists = np.squeeze(np.where(dists< pos_dist + margin))
            if len(semihard_dists):
                neg_cand = random.choice(semihard_dists)
            else:
                neg_cand = neg
            triplet["negative"] = reverse_mapping[neg_cand]
            new_triplets.append(triplet)
        
        self.triplets = new_triplets
    
    def state_dict(self):
        state = {}
        keys = ["photos_folder_name", "triplets","retrieval"]
        for k in keys:
            state[k] = getattr(self,k)
        return state
    
    @classmethod
    def load(cls, state_dict, transform):
        instance = cls(photos_folder_name=state_dict["photos_folder_name"], transform=transform)
        instance.retrieval = state_dict["retrieval"]
        instance.triplets = state_dict["triplets"]
        return instance
                                                     
    @classmethod
    def get_parsed_dataset(cls, triplets_file_name, photos_folder_name=PHOTO_FOLDER, 
                 retrieval_file_name=DRESS_FOLDER+ID_NAME, transform = None):
        ds = DressDataset(photos_folder_name=photos_folder_name, transform=transform)
        ds.parse_triplets(triplets_file_name=triplets_file_name, retrieval_file_name=retrieval_file_name)
        return ds  
        
        

In [50]:
fake_train = DressDataset.get_parsed_dataset(triplets_file_name=DRESS_FOLDER+TRAIN_NAME,
                     transform=torch_transform.Compose([
                                               torch_transform.ToTensor()]))

In [152]:
%%time
fake_mappings = {}
counter=0
for triplet in fake_train.triplets:
            pos, anc, neg = triplet["positive"], triplet["anchor"], triplet["negative"]
            if pos not in fake_mappings:
                fake_mappings[pos] = counter
                counter = counter+1
            if anc not in fake_mappings:
                fake_mappings[anc] = counter
                counter = counter+1
            if neg not in fake_mappings:
                fake_mappings[neg] = counter
                counter = counter+1
fake_embeddings = torch.randn(size=(len(fake_mappings),FINAL_EMBEDDING_SIZE)).cpu().data.numpy()
fake_train.resample(fake_embeddings, fake_mappings, margin=0.3)

/home/snake/anaconda3/envs/myenv/lib/python3.6/site-packages/numpy/linalg/linalg.py:2378: RuntimeWarning: invalid value encountered in sqrt
  return sqrt(add.reduce(s, axis=axis, keepdims=keepdims))


CPU times: user 1min 37s, sys: 1.03 s, total: 1min 38s
Wall time: 26.4 s


In [104]:
len(fake_mappings)

13980

In [103]:
fake_reverse_mapping = {v:k for k,v in six.iteritems(fake_mappings)}
fake_reverse_mapping[15600]

KeyError: 15600

In [61]:
train = DressDataset.get_parsed_dataset(triplets_file_name=DRESS_FOLDER+TRAIN_NAME,
                     transform=torch_transform.Compose([
                                               torch_transform.ToTensor()
                                           ]))
test = DressDataset.get_parsed_dataset(triplets_file_name=DRESS_FOLDER+TRAIN_NAME,
                   transform=torch_transform.Compose([
                                               torch_transform.ToTensor()
                                           ]))

In [62]:
micro_train  = train.get_sliced_copy(0,500)

In [63]:
VALIDATION_PROPORTION = 0.5
val_prop = int(len(test)*VALIDATION_PROPORTION)
validation = test.get_sliced_copy(0, val_prop)
final_test = test.get_sliced_copy(val_prop,len(test))

In [64]:
import torch

BATCH_SIZE = 42

has_cuda = torch.cuda.is_available()
kwargs = {'num_workers': 4, 'pin_memory': True} if has_cuda else {}
train_dataset_loader = torch.utils.data.DataLoader(train,batch_size=BATCH_SIZE, shuffle=True,**kwargs)
test_dataset_loader = torch.utils.data.DataLoader(test,batch_size=BATCH_SIZE, shuffle=True,**kwargs)

In [65]:
micro_train_loader = torch.utils.data.DataLoader(micro_train,batch_size=BATCH_SIZE, shuffle=True,**kwargs)

<h1>define the model</h1>

In [14]:
import torch.nn as nn
import torchvision.models as models


class BasisNet(nn.Module):
    def __init__(self, embedding_size=FINAL_EMBEDDING_SIZE):
        super(BasisNet, self).__init__()
        self.inception = models.inception_v3(pretrained=True)
        for param in self.inception.parameters():
            param.requires_grad=False
        num_ftrs = self.inception.fc.in_features
        self.inception.fc = nn.Linear(num_ftrs, 512)
        self.elu = nn.ELU()
        self.final = nn.Linear(512, embedding_size)
    
    def forward(self,x):
        x = self.inception(x)[0]
        x = self.elu(x)
        x = self.final(x)
        return self.l2_norm(x)
    
    def l2_norm(self,inp):
        input_size = inp.size()
        buffer = torch.pow(inp, 2)

        normp = torch.sum(buffer, 1).add_(1e-10)
        norm = torch.sqrt(normp)

        _output = torch.div(inp, norm.view(-1, 1).expand_as(inp))

        output = _output.view(input_size)

        return output
        
class TripletNet(nn.Module):
    def __init__(self, embedding_net):
        super(TripletNet, self).__init__()
        self.embedding_net = embedding_net

    def forward(self, positive, anchor, negative):
        output1 = self.embedding_net(positive)
        output2 = self.embedding_net(anchor)
        output3 = self.embedding_net(negative)
        return output1, output2, output3

    def get_embedding(self, x):
        return self.embedding_net(x)

In [37]:
cuda = torch.device('cuda') 

In [16]:
def loss_func(anchor, positive, negative, size_average=True, margin = 0.3):
    distance_positive = (anchor - positive).pow(2).sum(1)  # .pow(.5)
    distance_negative = (anchor - negative).pow(2).sum(1)  # .pow(.5)
    losses = F.relu(distance_positive - distance_negative + margin)
    return losses.mean() if size_average else losses.sum()

class TripletLoss(nn.Module):
    """
    Triplet loss
    Takes embeddings of an anchor sample, a positive sample and a negative sample
    """

    def __init__(self, margin):
        super(TripletLoss, self).__init__()
        self.margin = margin

    def forward(self, anchor, positive, negative, size_average=True):
        return loss_func(anchor, positive, negative, size_average, self.margin)
    
dress_net = TripletNet(BasisNet())
dress_net.to(cuda)

TripletNet(
  (embedding_net): BasisNet(
    (inception): Inception3(
      (Conv2d_1a_3x3): BasicConv2d(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      )
      (Conv2d_2a_3x3): BasicConv2d(
        (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      )
      (Conv2d_2b_3x3): BasicConv2d(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      )
      (Conv2d_3b_1x1): BasicConv2d(
        (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      )
      (Conv2d_4a_3x3): BasicConv2d(
        (conv):

In [17]:
def save_embedding(mapping, embeddings, emb_to_save, index):
    if index not in mapping:
        embeddings.append(emb_to_save)
        mapping[index] = len(embeddings) - 1

In [122]:
import torch.optim as optim

optimizer = optim.Adam(filter(lambda p: p.requires_grad,dress_net.parameters()), lr=0.0001)
 
all_losses =[]
criterion = TripletLoss(MARGIN)
for i in range(EPOCH):
    losses = []
    train_embeddings=[]
    train_embedding_map = {}
    torch.set_grad_enabled(False)
    for batch_i, (positive, anchor, negative) in enumerate(train_dataset_loader):
        vpositive, vanchor, vnegative = Variable(positive[0]).cuda(), Variable(anchor[0]).cuda(), Variable(negative[0]).cuda()
        p_emb,a_emb,n_emb = dress_net(vanchor, vpositive, vnegative)
        for i in range(len(p_emb)):
            save_embedding(train_embedding_map, train_embeddings, p_emb[i],positive[1][i].item())
            save_embedding(train_embedding_map, train_embeddings, a_emb[i],anchor[1][i].item())
            save_embedding(train_embedding_map, train_embeddings, n_emb[i],negative[1][i].item())
    print("started resampling")
    train_embeddings_ct = torch.stack(train_embeddings).cpu().data.numpy()
    train.resample(train_embeddings_ct, train_embedding_map,
                   margin = MARGIN)
    torch.set_grad_enabled(True)
    for batch_i, (positive, anchor, negative) in enumerate(train_dataset_loader):
        vpositive, vanchor, vnegative = Variable(positive[0]).cuda(), Variable(anchor[0]).cuda(), Variable(negative[0]).cuda()
        p_emb,a_emb,n_emb = dress_net(vanchor, vpositive, vnegative)
        loss = criterion(p_emb,a_emb,n_emb)
        losses.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    score = np.mean(losses)
    print("loss %s after epoch %s "%(score, epoch))
    all_losses.append(score)
    if epoch % 10 == 0:
        save_checkpoint({
            'epoch': epoch + 1,
            'model': dress_net.state_dict(),
            'dataset': micro_train.state_dict(),
            'optimizer' : optimizer.state_dict(),
        })

> <ipython-input-122-1afaf4aef17c>(13)<module>()->None
-> torch.set_grad_enabled(False)
(Pdb) n
> <ipython-input-122-1afaf4aef17c>(14)<module>()->None
-> for batch_i, (positive, anchor, negative) in enumerate(train_dataset_loader):
(Pdb) n
> <ipython-input-122-1afaf4aef17c>(15)<module>()->None
-> vpositive, vanchor, vnegative = Variable(positive[0]).cuda(), Variable(anchor[0]).cuda(), Variable(negative[0]).cuda()
(Pdb) p train_embeddings
[]
(Pdb) n
> <ipython-input-122-1afaf4aef17c>(16)<module>()->None
-> p_emb,a_emb,n_emb = dress_net(vanchor, vpositive, vnegative)
(Pdb) n
> <ipython-input-122-1afaf4aef17c>(17)<module>()->None
-> for i in range(len(p_emb)):
(Pdb) n
> <ipython-input-122-1afaf4aef17c>(18)<module>()->None
-> save_embedding(train_embedding_map, train_embeddings, p_emb[i],positive[1][i].item())
(Pdb) n
> <ipython-input-122-1afaf4aef17c>(19)<module>()->None
-> save_embedding(train_embedding_map, train_embeddings, a_emb[i],anchor[1][i].item())
(Pdb) n
> <ipython-input-122-1af

137 before resampling
138 before resampling
139 before resampling
140 before resampling
141 before resampling
142 before resampling
143 before resampling
144 before resampling


/home/snake/anaconda3/envs/myenv/lib/python3.6/site-packages/PIL/Image.py:2546: DecompressionBombWarning: Image size (123871510 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)


145 before resampling
146 before resampling
147 before resampling
148 before resampling
149 before resampling
150 before resampling
151 before resampling
152 before resampling
153 before resampling
154 before resampling
155 before resampling
156 before resampling
157 before resampling
158 before resampling
159 before resampling
160 before resampling
161 before resampling
162 before resampling
163 before resampling
164 before resampling
165 before resampling
166 before resampling
167 before resampling
168 before resampling
169 before resampling
170 before resampling
171 before resampling
172 before resampling
173 before resampling
174 before resampling
175 before resampling
176 before resampling
177 before resampling
178 before resampling
179 before resampling
180 before resampling
181 before resampling
182 before resampling
183 before resampling
184 before resampling
185 before resampling
186 before resampling
187 before resampling
188 before resampling
189 before resampling
190 before

AxisError: axis 1 is out of bounds for array of dimension 1

In [85]:
len(train_embeddings)

887

In [70]:
def save_checkpoint(state, filename=CHECKPOINT_FILE):
    torch.save(state, filename)


In [ ]:
import torch.optim as optim

optimizer = optim.Adam(filter(lambda p: p.requires_grad,dress_net.parameters()), lr=0.0001)

criterion = TripletLoss(MARGIN)
for epoch in range(3):
    losses = []
    train_embeddings=[]
    train_embedding_map = {}
    torch.set_grad_enabled(False)
    for batch_i, (positive, anchor, negative) in enumerate(train_dataset_loader):
        vpositive, vanchor, vnegative = Variable(positive[0]).cuda(), Variable(anchor[0]).cuda(), Variable(negative[0]).cuda()
        p_emb,a_emb,n_emb = dress_net(vanchor, vpositive, vnegative)
        for i in range(len(p_emb)):
            save_embedding(train_embedding_map, train_embeddings, p_emb[i],positive[1][i].item())
            save_embedding(train_embedding_map, train_embeddings, a_emb[i],anchor[1][i].item())
            save_embedding(train_embedding_map, train_embeddings, n_emb[i],negative[1][i].item())
    train_embeddings_ct = torch.stack(train_embeddings).cpu().data.numpy()
    micro_train.resample(train_embeddings_ct, train_embedding_map,
                   margin = MARGIN)
    torch.set_grad_enabled(True)
    for batch_i, (positive, anchor, negative) in enumerate(train_dataset_loader):
        vpositive, vanchor, vnegative = Variable(positive[0]).cuda(), Variable(anchor[0]).cuda(), Variable(negative[0]).cuda()
        p_emb,a_emb,n_emb = dress_net(vanchor, vpositive, vnegative)
        loss = criterion(p_emb,a_emb,n_emb)
        losses.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    score = np.mean(losses)
    print("loss %s after epoch %s "%(score, epoch))
    if epoch % 50 == 0:
        save_checkpoint({
            'epoch': epoch + 1,
            'model': dress_net.state_dict(),
            'dataset': micro_train.state_dict(),
            'optimizer' : optimizer.state_dict(),
        })
    

/home/snake/anaconda3/envs/myenv/lib/python3.6/site-packages/PIL/Image.py:2546: DecompressionBombWarning: Image size (123871510 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  DecompressionBombWarning)
/home/snake/anaconda3/envs/myenv/lib/python3.6/site-packages/numpy/linalg/linalg.py:2378: RuntimeWarning: invalid value encountered in sqrt
  return sqrt(add.reduce(s, axis=axis, keepdims=keepdims))


In [67]:
saved_net = TripletNet(BasisNet())
saved_net.to(cuda)
saved_optimizer = optim.Adam(filter(lambda p: p.requires_grad,saved_net.parameters()), lr=0.0001)
checkpoint = torch.load('/tmp/checkpoint.pth.tar')
saved_model = saved_net.load_state_dict(checkpoint["model"])
saved_optimizer = saved_optimizer.load_state_dict(checkpoint["optimizer"])
saved_ds = DressDataset.load(checkpoint["dataset"], transform = train.transform)

In [68]:
saved_ds.triplets

[{'bbox': {'width': 1605, 'top': 1472, 'left': 1612, 'height': 2189},
  'positive': 27273,
  'anchor': 7683,
  'negative': 7864},
 {'bbox': {'width': 229, 'top': 301, 'left': 445, 'height': 421},
  'positive': 36288,
  'anchor': 9251,
  'negative': 4210},
 {'bbox': {'width': 1427, 'top': 1214, 'left': 837, 'height': 1898},
  'positive': 97856,
  'anchor': 6475,
  'negative': 35252},
 {'bbox': {'width': 411, 'top': 343, 'left': 322, 'height': 741},
  'positive': 97325,
  'anchor': 1576,
  'negative': 11621},
 {'bbox': {'width': 200, 'top': 104, 'left': 207, 'height': 292},
  'positive': 47060,
  'anchor': 16427,
  'negative': 589},
 {'bbox': {'width': 528, 'top': 249, 'left': 119, 'height': 856},
  'positive': 69973,
  'anchor': 8956,
  'negative': 57237},
 {'bbox': {'width': 689, 'top': 569, 'left': 550, 'height': 1415},
  'positive': 71822,
  'anchor': 2701,
  'negative': 97994},
 {'bbox': {'width': 1021, 'top': 1547, 'left': 1477, 'height': 475},
  'positive': 45742,
  'anchor': 8786